In [1]:
import chromedriver_autoinstaller

In [4]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [5]:
youtube = []
driver = webdriver.Chrome()
url = "https://www.youtube.com/results?search_query=artificial+intelligence"
driver.get(url)
options = webdriver.ChromeOptions()
options.add_argument("--incognito")

wait = WebDriverWait(driver, 10)
cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
cookie_consent.click()

#while True:
#    scroll_height = 10000
#    document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
#    driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
#    time.sleep(3)
#    document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
#    if document_height_after == document_height_before:
#        break

# Extract the HTML and parse it with BeautifulSoup
html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, "html.parser")

video_name = soup.find("yt-formatted-string", class_="style-scope ytd-video-renderer")
video_title = video_name.getText().strip()
video_link = [a.get('href') for a in soup.find_all('a', class_= 'yt-simple-endpoint style-scope ytd-video-renderer', href=True)]
#video_views = soup.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-bloc")
#video_upload = soup.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-bloc")



# Zip to add all values in a tuple for each restaurant
rows = list(zip(video_title))
#, video_link, video_views, video_upload))
youtube.extend(rows)
        
df_yt = pd.DataFrame(youtube, columns=['Name'])



In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re


In [90]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [91]:
search_youtube_videos('artificial intelligence')

,Name,views,duration,date,description,link
0,What Is Artificial Intelligence? | 🔥Artificial...,"1,162,491","5 minutes, 28 seconds",3 years ago,"This video on ""What is Artificial Intelligence...",https://www.youtube.com/watch?v=ad79nYk2keg
1,How will AI change the world?,"717,034","5 minutes, 56 seconds",3 months ago,Explore the current limitations of artificial ...,https://www.youtube.com/watch?v=RzkD_rTEBYs
2,Steven Spielberg Talks Artificial Intelligence...,"52,643","3 minutes, 37 seconds",1 day ago,In this bonus scene from Stephen Colbert's int...,https://www.youtube.com/watch?v=tSoCtw5vNS0
3,KSP: Conquering Jool using Artificial Intellig...,"97,979",Unknown,22 hours ago,Inspired by a reddit post and the recent uptic...,https://www.youtube.com/watch?v=OEg6XuXVsdM
4,"Generative AI Is About To Reset Everything, An...","519,004",Unknown,2 weeks ago,"Whether you like it or not, generative AI like...",https://www.youtube.com/watch?v=WY518YRfs5M
...,...,...,...,...,...,...
617,How Google is Losing the Generative AI Race an...,unknown,"4 minutes, 18 seconds",7 hours ago,Generative AI is a technology that can produce...,https://www.youtube.com/watch?v=zD3PbtwC3Eg
618,AI in Education Enhancing Learning with Machin...,20,Unknown,14 hours ago,"In this video, we explore the transformative i...",https://www.youtube.com/shorts/5hnM57hSa6M
619,Maximize Your Learning Efficiency with These A...,4,"1 minute, 46 seconds",2 days ago,Are you struggling to keep up with your studie...,https://www.youtube.com/watch?v=r9F0hVH2MyM
620,AI Robots will Replace Humans? Figure 01 New H...,unknown,Unknown,Unknown,Artificial Intelligence and robotics has the a...,https://www.youtube.com/shorts/jpJHlYEPh-s


In [43]:
df_yt.head(10)

,Name,views,duration,date,link
0,What Is Artificial Intelligence? | 🔥Artificial...,"1,162,169","5 minutes, 28 seconds",3 years ago,https://www.youtube.com/watch?v=ad79nYk2keg
1,Artificial intelligence and algorithms: pros a...,"10,008,090",Unknown,3 years ago,https://www.youtube.com/watch?v=s0dMTAQM4cw
2,Steven Spielberg Talks Artificial Intelligence...,"51,522","3 minutes, 37 seconds",1 day ago,https://www.youtube.com/watch?v=tSoCtw5vNS0
3,KSP: Conquering Jool using Artificial Intellig...,"94,379",Unknown,20 hours ago,https://www.youtube.com/watch?v=OEg6XuXVsdM
4,The Danger of AI | Scary Technology | Artifici...,"68,115",Unknown,2 weeks ago,https://www.youtube.com/watch?v=5guiUrgcblM
5,"Generative AI Is About To Reset Everything, An...","518,484",Unknown,2 weeks ago,https://www.youtube.com/watch?v=WY518YRfs5M
6,Artificial Intelligence: Last Week Tonight wit...,"5,134,141",Unknown,10 days ago,https://www.youtube.com/watch?v=Sqa8Zo2XWc4
7,How will AI change the world?,"716,036","5 minutes, 56 seconds",3 months ago,https://www.youtube.com/watch?v=RzkD_rTEBYs
8,"ChatGPT: Artificial Intelligence, chatbots and...","159,460","13 minutes, 22 seconds",2 days ago,https://www.youtube.com/watch?v=1wzPr4cUoMQ
9,Rehearsals underway for artificial intelligenc...,249,"1 minute, 42 seconds",14 hours ago,https://www.youtube.com/watch?v=7g_Vgfb-3nE


,Name,views,duration,date,link
0,Elon Musk on Artificial Intelligence Implicati...,"18,836",Unknown,3 months ago,https://www.youtube.com/watch?v=sAUpoJixwZg
1,learning AI and ChatGPT isn’t that hard,"193,890",Unknown,7 days ago,https://www.youtube.com/watch?v=JJCq21Dc-Us


In [70]:
def scrape_comments(df):
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)

    rows = []
    for i in range(len(df)):
        url = df['link'][i]
        driver.get(url)
        time.sleep(30)
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(5)

        while True:
            scroll_height = 10
            document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
            time.sleep(5)
            document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
            if document_height_after == document_height_before:
                break

        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, "html.parser")
            
        comments = soup.find_all("div", id="main", class_="style-scope ytd-comment-renderer")

        for comment in comments:
            username = comment.find("span", class_="style-scope ytd-comment-renderer").text.strip()
            username = username.replace('\n', '')
            yt_comment = comment.find("yt-formatted-string", class_="style-scope ytd-comment-renderer").text
            likes = comment.find("span", class_="style-scope ytd-comment-action-buttons-renderer").text.strip()
            date = comment.find("yt-formatted-string", class_="published-time-text style-scope ytd-comment-renderer").text.strip()
            date = date.replace('(edited)', '')
            link = df['link'][i]
            
            rows.append([username, yt_comment, likes, date, link])

    df_yt_comment = pd.DataFrame(rows, columns=['Username', 'Comment', 'likes', 'date', 'link'])

    driver.quit()

    return df_yt_comment


In [71]:
scrape_comments(df_yt)

,Username,Comment,likes,date,link
0,@MrAndrew535,"The term ""Artificial Intelligence"" has two com...",4,3 months ago,https://www.youtube.com/watch?v=sAUpoJixwZg
1,@debraboaze2991,Elon & Joe Rogan so fun together.,5,3 months ago,https://www.youtube.com/watch?v=sAUpoJixwZg
2,@infinidimensionalinfinitie5021,seems to me that this singularity happened an ...,1,2 weeks ago,https://www.youtube.com/watch?v=sAUpoJixwZg
3,@brainwashedbyevidence948,"I am designing my own ai, at a very strong ai ...",0,12 days ago,https://www.youtube.com/watch?v=sAUpoJixwZg
4,@julietpalmer9294,Good luck to you in all your endevors,0,23 hours ago,https://www.youtube.com/watch?v=sAUpoJixwZg
...,...,...,...,...,...
261,@no-eb2xx,I'd rather learn how to destroy Skynet.,2,7 days ago,https://www.youtube.com/watch?v=JJCq21Dc-Us
262,@khanhnguyenngoc2202,no you don't,0,7 days ago,https://www.youtube.com/watch?v=JJCq21Dc-Us
263,@nonatobk,"those dudes are boring..,",0,6 days ago,https://www.youtube.com/watch?v=JJCq21Dc-Us
264,@JadenJahci,Minoe,0,7 days ago,https://www.youtube.com/watch?v=JJCq21Dc-Us


In [83]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'